# Calculating thermal time indices and merging UAV data.

**Name**: Byron Evers<br/>
KSU Wheat Genetics Resource Center

## Table of contents
1. [Motivation](#BJE_motivation)
2. [Objective](#BJE_objective)
3. [Equations](#BJE_equations)
    * [GDD](#BJE_GDD)
    * [Pdays](#BJE_Pdays)
    * [BMT](#BJE_BMT)
4. [Project Steps](#BJE_examples)
    * [Step 1: Import Modules](#BJE_step1)
    * [Step 2: Download Data](#BJE_step2)
    * [Step 3: Format Data Frame](#BJE_step3)
    * [Step 4: Define User Inputs](#BJE_step4)
    * [Step 5: Calculate Photoperiod](#BJE_step5)
    * [Step 6: Define Growing Season](#BJE_step6)
    * [Step 7: Calculate GDD](#BJE_step7)
    * [Step 8: Calculate Pdays ](#BJE_step8)
    * [Step 9: Calculate BMT](#BJE_BMTe)
    * [Step 10: Graph Results](#BJE_step10)
    * [Step 11: Merge UAV and Thermal Time Index](#BJE_step11)
5. [Data Usage Policy](#BJE_policy)
6. [References](#BJE_ref)

<a name="BJE_motivation"></a>

## Motivation
- Poland labs current UAV pipeline includes stitching photos and extracting plot level reflectance data through Agisoft software.
- Data received from this process is in either an Excel or csv file. 
- The data set includes reflectance values for 5 individual bands (R,G,B,RE and NIR) and 3 vegetative indices (NDVI, NDRE and GNDVI). 
- Currently a wide range of time scales are used for UAV data analysis. 
    * Days after planting
    * Day of Year
    * Thermal Time
        * GDD
        * Pdays
        * BMT
        
Thermal time indices are important for data analysis between years. However, how thermal time is calculated can also influence reflectance data. Using set values such as a base temperature, maximum temperature and optimum temperature all effect thermal time. Additionally, photo period can also affect the growing stage in many crops. Typically, wheat is not as photo period dependent but in crops such as soybeans it can heavily influence crop development. For this project three thermal time indices were chosen to further explore the differences between these indices.

<a name="BJE_objective"></a>
## Objective

1. Calculate thermal time indices for all UAV collection dates
    * growing degree days (GDD)
    * physiological days (Pdays)
    * biometeorological time (BMT) 
    * Wang and Engel Degree Days  $GDD_{WE}$
    * Photo Degree Days (PGDD)
2. Merge all of the UAV reflectance data, plot level phenotypic data and the calculated thermal values into one .csv file.


<a name="BJE_equations"></a>
## Equations

<a name="BJE_GDD"></a>
### Growing Degree Days (GDD):
$$GDD = \sum_{Planting}^{Harvest}(\frac{Tmax+Tmin}{2})-Tbase$$

<a name="BJE_Pdays"></a>
### Physiological Days (Pdays):
$$Pdays = \frac{1}{24}(5*P(T_1)+8*P(T_2)+8*P(T_3)+3*P(T_4))$$
**Where**
* $T_1=Tmin$
* $T_2=\frac{(2*Tmin)+Tmax}{3}$
* $T_3=\frac{Tmin+(2*Tmax)}{3}$
* $T_4=Tmax$

**And P is**
* $P=0$   When $T <=Tmin$
* $P=k*(1-\frac{(T-Topt)2}{(Topt-Tmin)2})$ when $Tmin <= T <=Topt$
* $P=k*(1-\frac{(T-Topt)2}{(Tmax-Topt)2})$ when $Topt <= T <=Tmax$
* $P=0$ when $T >=Tmax$

**And k is**
* $P=0$   When $T <=Tmin$

<a name="BJE_BMT"></a>
### Biometeorological Time
$$BMT = \sum_{Planting}^{Harvest}[a_1(L-a_0) + a_2(L-a_0)^2* {[b_1(Tmax-b_0) + b_2(Tmax-b_0)^2}+{d_1(Tmin-b_0) + d_2(Tmin-b_0)^2]}$$
or for simplicity:
$$BMT = \sum_{Planting}^{Harvest}V_1 {(V_2}+{V_3)}$$
**Where**
* $L$= daily photoperiod
* $a_0$= base daylength
* $b_0$= base temperature
* $a_1, a_2, b_1, b_2, d_1, d_2$ are response coefficients
    * For this project I used the coefficents detailed by Robertson (1968)

### Wang and Engel 1998 Growing Degree Days ($GDD_{WE}$):
$$\alpha= (\frac{ln(2)}{ln(\frac{Tmax-Tmin}{Topt-Tmin})})$$

$$ Numerator= 2(Tav-Tmin)^{\alpha}(Topt-Tmin)^{\alpha}-2(Tav-Tmin)^{\alpha}$$

$$ Dnominator = (Topt-Tmin)^{2\alpha}$$

$$WEDD = \sum_{Planting}^{Harvest}(\frac{Numerator}{Denominator})-(Topt-Tmin)$$

### Photo Growing Degree Days (PGDD):

$$PGDD = (WEDD)((1-0.002\times2){\times(20-day\ length))}^2$$

<a name="BJE_examples"></a>
## Examples and Operational Code

Thank you for reviewing my project and I appreciate any feedback you can provide. This is a fully functional version that pulls data from KSU Mesonet. If for some reason KSU Mesonet is not working I do have a .csv file with weather data save in the data folder. 

<a name="#BJE_step1"></a>

# Step 1: Import needed modules
To run ephm you may need to install it first. To do this please run **pip install ephem** command line or execute the cell bellow.

In [83]:
!pip install ephem
!pip install numpy


distutils: /Users/bevers/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/bevers/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /Users/bevers/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/bevers/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/Users/bevers/anaconda3/bin/python -m pip install --upgrade pip' command.
distutils: /Users/bevers/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/bevers/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /Users/bevers/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/bevers/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/Users/bevers/anaconda3/bin/python -m pip install --upgrade pip' command.


In [84]:
import pandas as pd
import numpy as np
import datetime
import ephem
import matplotlib.pyplot as plt
%matplotlib inline

<a name="BJE_step2"></a>
# Step 2: Download Data from the KSU Mesonet. 
* For this project I am only interested in day interval and there for it is hard coded in to the URL
* Users should edit the stn variable to the station they desire. A list of stations is provided at; http://mesonet.k-state.edu/rest/stationnames/
* Date searches must be formatted as YYYYmmddHHMMSS (year, month, day, hour, minute, second). 
* Currently the start date is 20140101000000, or Jan 1, 2014. Granted that is well beyond the scope of this project, is over kill and may slow the download process. However, I used this to demonstrate the power and time scale flexibility of this project. 
* Similar to the start time the end time, as it is set to the time the user executes the cell, is potentially not useful. However, again here I wanted to demonstrate another way to create inputs for the URL builder.
* In sequential steps the large data frame will reduces to capture only date times of interest.
* Additional documentation can be found at; http://mesonet.k-state.edu/rest/

<a name="Update__inputs"></a>

In [85]:
# Set connection to KS Mesonet
root = 'http://mesonet.k-state.edu/rest/stationdata/?'
stn = 'Hutchinson%20'
sub='10SW'
start_time = "20190601000000" 
end_time ="20200701000000" 



In [86]:
# Create url to mesonet
url = root + "stn="+stn+sub+'&int=day'+'&t_start='+start_time+'&t_end='+end_time

#url2 = root + "stn="+stn2+'&int=day'+'&t_start='+start_time2+'&t_end='+end_time2
url #prints url so user can check format

'http://mesonet.k-state.edu/rest/stationdata/?stn=Hutchinson%2010SW&int=day&t_start=20190601000000&t_end=20200701000000'

<span style="color:red">**The cell below will pull data from KSU Mesonet. This works as long as the Mesonet site is operational. Additionally, this can be slow depending on your internet connection, feel free to access data through the URL. However, after the initial download I recommend saving the data as a .txt so that it can be accessed from your local device in the future.**<span>

In [90]:
df = pd.read_csv(url)
print(type(df))
df.head(2)

<class 'pandas.core.frame.DataFrame'>


,TIMESTAMP,STATION,PRESSUREAVG,PRESSUREMAX,PRESSUREMIN,SLPAVG,TEMP2MAVG,TEMP2MMIN,TEMP2MMAX,TEMP10MAVG,...,SOILPA20CM,SOILPA50CM,SOILVR5CM,SOILVR10CM,SOILVR20CM,SOILVR50CM,VWC5CM,VWC10CM,VWC20CM,VWC50CM
0,2019-06-01 00:00:00,Hutchinson 10SW,95.98,96.21,95.75,101.32,21.34,13.61,28.66,21.93,...,1.8847,1.9624,1.5845,1.6404,1.7449,1.8507,0.37457,0.37793,0.41093,0.44704
1,2019-06-02 00:00:00,Hutchinson 10SW,95.83,96.03,95.66,101.13,22.79,15.11,30.42,23.37,...,1.8819,1.9568,1.5732,1.6392,1.7559,1.8580,0.36805,0.37461,0.40906,0.44421


**df should now be a pandas.core.frame.DataFrame if not please try to re-upload the data either from the url or if you previously saved the data directly from the data folder. If you cannot get the data to load properly please contact me.**

<a name="BJE_step3"></a>
# Step 3: Format Data Frame 
* For this project the only columns needs are;
    * Date
    * Station
    * Min daily temperature
    * Max daily temperature
* The cell below selects those columns, renames the columns, interpolates data, reformats the data to the types needed and then shows a 5 row preview of the data. 

In [91]:
# Select colums you want and put in a dataframe
df =df[['TIMESTAMP', 'STATION', 'TEMP2MMIN','TEMP2MMAX']] #selects columns needed
df.rename(columns={'TIMESTAMP':'Date', 'TEMP2MMAX':'Tmax', 'TEMP2MMIN':'Tmin'}, inplace=True) #renames columns 
df['Date'] =  pd.to_datetime(df['Date'],format='%Y-%m-%d')
#df['DOY'] = df.Date.strftime('%j')

print(type(df.Tmin[2]))
print(type(df.STATION[2]))
df.head(5)


<class 'numpy.float64'>
<class 'str'>


,Date,STATION,Tmin,Tmax
0,2019-06-01,Hutchinson 10SW,13.61,28.66
1,2019-06-02,Hutchinson 10SW,15.11,30.42
2,2019-06-03,Hutchinson 10SW,17.14,28.52
3,2019-06-04,Hutchinson 10SW,18.77,29.74
4,2019-06-05,Hutchinson 10SW,17.96,31.44


In [92]:
print(type(df.Tmax[4]))

<class 'numpy.float64'>


**For missing data some times Mensonet uses NaN and other times uses M. Regardless of the designation the cell below removes these values**

In [93]:
df[pd.to_numeric(df.Tmax, errors='coerce').isnull()]
df=df[pd.to_numeric(df.Tmax, errors='coerce').notnull()]

In [94]:
df.Tmin = df.Tmin.astype(float)
df.Tmax = df.Tmax.astype(float)
df = df.interpolate()
df.head(2)

,Date,STATION,Tmin,Tmax
0,2019-06-01,Hutchinson 10SW,13.61,28.66
1,2019-06-02,Hutchinson 10SW,15.11,30.42


<a name="BJE_step4"></a>
# Step 4: Define User Inputs
* The inputs needed are;
    * plantDate- this is to start the season of interest. This can be stored as a variable now or entered as a string in the "Growing_season" function that will be defined later. Format of input needs to be YYYY-mm-dd.
    * harvestDate- this is to end the season of interest. This can be stored as a variable now or entered as a string in the "Growing_season" function that will be defined later. Format of input needs to be YYYY-mm-dd.
    * long- this is a decimal degree longitude point for the field of interest. This is used by the ephem module to give the most accurate photoperiod. 
    * lat- this is a decimal degree latitude point for the field of interest. This is used by the ephem module to give the most accurate photoperiod. 
    * tbase- this sets the base temperature for the crop of interest in degrees C. In this case wheat is the crop of interest. Temperatures of 5C and 0C are acceptable but 5 is chosen for this example.
    * k- is a set scale factor for the Pdays equation. This maybe be edited for other crops. 
    * topt- this sets the optimumn growing temperature for the crop of interest in degrees C. For wheat in this example 17C was used.
    * tmax- this sets the maximum growing temperature for the crop of interest in degrees C. For wheat in this example 30C was used.
    
**All of these parameters are designed so that users can adjust them to fit the needs of the individual project**

In [45]:
# Define inputs 
plantDate=np.datetime64('2019-10-10') #set the date your crop was planted
harvestDate=np.datetime64('2020-06-22') #set the date your crop was harvest
# Set your location to be used for photoperiod
lat = '37.930970'
long = '-98.020000' 


# GDD inputs
tbase= 0 #set the base temperature for your given crop

# Pdays input
k= .35 #scale factor 
topt = 17 #optimal growing temperature for your given crop
tmax = 30 #maximum growing temperature for your given crop


# Step 5: Calculate Photoperiod

To complete this the ephem module is used. The output is displayed as decimal hours with the times of sunrise and sunset on a 24 hour clock. Date frame must have the following columns;
* Date
* lat
* long

In [46]:
obs = ephem.Observer()
obs.long = ephem.degrees(long)
obs.lat = ephem.degrees(lat)
date =  df['Date'].dt.strftime('%Y/%m/%d')
sun = ephem.Sun(obs)

rise_data=[]
set_data=[]
photoperiod_data=[]

for i in date:
    obs.date = ephem.Date(i)
    r1 = obs.next_rising(sun)
    s1 = obs.next_setting(sun)
    r1_lt = ephem.Date(r1 - 5 * ephem.hour) #local time 
    (y, mn, d, h, min, s) = r1_lt.tuple()
    rise = ( h + min/60. + s/3600. )
    s1_lt = ephem.Date(s1 - 5 * ephem.hour) #local time
    (y, mn, d, h, min, s) = s1_lt.tuple()
    sunset = ( h + min/60. + s/3600. )
    photoperiod=sunset-rise
    rise_data.append(rise)
    set_data.append(sunset)
    photoperiod_data.append(photoperiod)
    df['Sunrise']= pd.DataFrame(rise_data)
    df['Sunset']= pd.DataFrame(set_data)
    df['PhotoPeriod']= pd.DataFrame(photoperiod_data)   
df.head(5)    


,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod
0,2019-06-01,Hutchinson 10SW,13.61,28.66,6.186724,20.804979,14.618255
1,2019-06-02,Hutchinson 10SW,15.11,30.42,6.180718,20.816324,14.635606
2,2019-06-03,Hutchinson 10SW,17.14,28.52,6.175216,20.827385,14.652169
3,2019-06-04,Hutchinson 10SW,18.77,29.74,6.170216,20.838149,14.667933
4,2019-06-05,Hutchinson 10SW,17.96,31.44,6.165719,20.848604,14.682885


# <a name="BJE_step6"></a>
# Step 6: Define Growing Season
The function will only return dates that fall within the growing season of interest. When running the function multiple 'sub-data frames" can be created at once for each growing season of interest. As long as the planting comes before the harvest and the dates are within the limits of the data downloaded from the Mesonet any combination will work. If the selected dates are beyond the limits of the data set the function bellow will return an error. When running this example, I encourage the user to edit the "plant" string date to '1984-10-13' to test the lower limit error. Likewise change the "harvest" string to '2222-06-21' to check the upper limit error.

In [47]:
def Growing_season (plant, harvest, df):
    """
    Sets the growing season form plant to harvest for each growing season 
   
     Input: Requires three inputs;
    1. Plant date string formatted as 'YYYY-MM-DD' this can be in the form of a stored variable or a direct string input
    2. Harvest date, string, formatted as 'YYYY-MM-DD' this can be in the form of a stored variable or a direct string input
    3. Pandas data frame with required date column formatted as 'YYYY-MM-DD'
    
    Output: Pandas data frame. With the dates of the growing season of interest. Recommend to save the ouput as a variable
        
    Byron Evers
    
    2019-03-06 
    """
    plant = np.datetime64(plant)
    harvest = np.datetime64(harvest)
    min_date = df.Date.min()
    min_date = min_date.to_datetime64()
    max_date = df.Date.max()
    max_date = max_date.to_datetime64()
    if min_date > plant:
        print('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if min_date > harvest:
        print('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if max_date < harvest:
        print('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if max_date < plant:
        print('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    df = df.drop(df[df.Date < plant].index)
    df = df.drop(df[df.Date > harvest].index)
    return df

In [51]:
#Call the function and store the data as a new data frame. In this example two data frames are created,df17 and df18, for the 2017 and 2018 winter wheat growing season
df20_1 = Growing_season('2019-10-10','2020-06-22',df)
df20= Growing_season(plantDate,harvestDate,df)
# For df20_1 date as a string was used where as df20 used the stored variable from input cell. 
#This is just to demonstrate that either format is accepted with this function.
df20.head(5)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod
131,2019-10-10,Hutchinson 10SW,12.80,25.13,7.590531,19.059878,11.469347
132,2019-10-11,Hutchinson 10SW,2.44,20.88,7.606029,19.035518,11.429489
133,2019-10-12,Hutchinson 10SW,-1.70,11.78,7.621613,19.011333,11.389720
134,2019-10-13,Hutchinson 10SW,-0.61,22.28,7.637286,18.963526,11.326240
135,2019-10-14,Hutchinson 10SW,0.07,23.03,7.653049,18.939922,11.286873


In [52]:
df20.tail(5)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod
383,2020-06-18,Hutchinson 10SW,21.21,34.97,6.153961,20.954935,14.800974
384,2020-06-19,Hutchinson 10SW,18.63,35.78,6.156722,20.959655,14.802932
385,2020-06-20,Hutchinson 10SW,17.21,21.49,6.159946,20.963920,14.803974
386,2020-06-21,Hutchinson 10SW,18.30,29.54,6.163625,20.967724,14.804099
387,2020-06-22,Hutchinson 10SW,16.74,31.44,6.167752,20.971058,14.803306


<a name="BJE_step7"></a>
# Step 7a: Calculate GDD- McMaster Method 1
The next cell defines a function to calculate growing degree days. This was developed for winter wheat with variable accumlation times, including full season, after Jan. 1 and after Mar. 1. For summer annual crops the Jan. 1 and Mar. 1 accumliation periods are not needed adn 

In [55]:
def GDD(df):
    """
    Calculates an individual and cumulative value for GDD for each day in the data frame
    
    Input: Pandas data frame. Minimum required columns include:
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        Date = date of collection
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        GDD= The growing degree day value for that individual day
        cum_GDD = The cumulative growing degree day value for all days in the designated growing period.
    
    Byron Evers
    
    2019-03-06
    """
    df['Tbase']=tbase
    tavg =((df.Tmax+df.Tmin)/2)
    values = np.where(tavg < tbase, df.Tbase, tavg).astype(float)
    df['GDD']=(values)-df.Tbase
    df['cum_GDD'] = df.GDD.cumsum()
    
    ### Delete between the # if you have a summer annual crop
    temp = df[df.Date.dt.month < 8]
    tavg2 =((temp.Tmax+temp.Tmin)/2)
    values2 = np.where(tavg2 < tbase, temp.Tbase, tavg2).astype(float)
    df['GDD_jan1']=(values2)-temp.Tbase
    df['cum_GDD_jan1'] = df.GDD_jan1.cumsum()
    
    temp2 = df[df.Date.dt.month >= 3]
    temp2 = temp2[temp2.Date.dt.month < 8]
    tavg3 =((temp2.Tmax+temp2.Tmin)/2)
    values3 = np.where(tavg3 < tbase, temp2.Tbase, tavg3).astype(float)
    df['GDD_mar1']=(values3)-temp2.Tbase
    df['cum_GDD_mar1'] = df.GDD_mar1.cumsum()
    ### Delete between the # if you have a summer annual crop
    
    return df

**Call the function and override the data frame so the new variables are added**

In [56]:
df = GDD(df)
df.head(2)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,GDD_jan1,cum_GDD_jan1,GDD_mar1,cum_GDD_mar1
0,2019-06-01,Hutchinson 10SW,13.61,28.66,6.186724,20.804979,14.618255,0,21.135,21.135,21.135,21.135,21.135,21.135
1,2019-06-02,Hutchinson 10SW,15.11,30.42,6.180718,20.816324,14.635606,0,22.765,43.900,22.765,43.900,22.765,43.900


<a name="BJE_step7"></a>
# Step 7b: Calculate GDD Method 2
The next cell defines a function to calculate growing degree days using method 2 as discribed in McMaster 1998

In [59]:
def GDD2(df):
    """
    Calculates an individual and cumulative value for GDD for each day in the data frame as descirbe by McMaster as method 2
    
    Input: Pandas data frame. Minimum required columns include:
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        Date = date of collection
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        GDD2= The growing degree day value for that individual day
        cum_GDD2 = The cumulative growing degree day value for all days in the designated growing period.
    
    Byron Evers
    
    2019-03-06
    """
    df['Tbase']=tbase
    tmax=np.where(df.Tmax<df.Tbase,df.Tbase,df.Tmax).astype(float)
    tmin=np.where(df.Tmin<df.Tbase,df.Tbase,df.Tmin).astype(float)
    values =((tmax+tmin)/2)
    df['GDD2']=(values)-df.Tbase
    df['cum_GDD2'] = df.GDD2.cumsum()
    
    ### Delete between the # if you have a summer annual crop
    temp = df[df.Date.dt.month < 8]
    tmax=np.where(temp.Tmax<temp.Tbase,temp.Tbase,temp.Tmax).astype(float)
    tmin=np.where(temp.Tmin<temp.Tbase,temp.Tbase,temp.Tmin).astype(float)
    values2 =((tmax+tmin)/2)
    df['GDD2_jan1']=(values2)-temp.Tbase
    df['cum_GDD2_jan1'] = df.GDD2_jan1.cumsum()
    
    temp2 = df[df.Date.dt.month >= 3]
    temp2 = temp2[temp2.Date.dt.month < 8]
    tmax=np.where(temp2.Tmax<temp2.Tbase,temp2.Tbase,temp2.Tmax).astype(float)
    tmin=np.where(temp2.Tmin<temp2.Tbase,temp2.Tbase,temp2.Tmin).astype(float)
    values3 =((tmax+tmin)/2)
    df['GDD2_mar1']=(values3)-temp2.Tbase
    df['cum_GDD2_mar1'] = df.GDD2_mar1.cumsum()
    ### Delete between the # if you have a summer annual crop
    
    return df

In [60]:
df = GDD2(df)
df.head(2)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,GDD_jan1,cum_GDD_jan1,GDD_mar1,cum_GDD_mar1,GDD2,cum_GDD2,GDD2_jan1,cum_GDD2_jan1,GDD2_mar1,cum_GDD2_mar1
0,2019-06-01,Hutchinson 10SW,13.61,28.66,6.186724,20.804979,14.618255,0,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135
1,2019-06-02,Hutchinson 10SW,15.11,30.42,6.180718,20.816324,14.635606,0,22.765,43.900,22.765,43.900,22.765,43.900,22.765,43.900,22.765,43.900,22.765,43.900


# Step 7c: Calculate GDD- McMaster Method 2 with a Tmax

In [62]:
def GDD3(df):
    """
    Calculates an individual and cumulative value for GDD for each day in the data frame as descirbe by McMaster as method 2
    
    Input: Pandas data frame. Minimum required columns include:
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        Date = date of collection
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        GDD2= The growing degree day value for that individual day
        cum_GDD2 = The cumulative growing degree day value for all days in the designated growing period.
    
    Byron Evers
    
    2021-04-26
    """
    df['Tbase']=tbase
    
    tmax=np.where(df.Tmax<0,0,np.where(df.Tmax>30,30,df.Tmax)).astype(float)
    tmin=np.where(df.Tmin<0,0,np.where(df.Tmin>30,30,df.Tmin)).astype(float)
    
    values =((tmax+tmin)/2)
    df['Tmax_GDD']=(values)-df.Tbase
    df['cum_Tmax_GDD'] = df.Tmax_GDD.cumsum()
    
    temp = df[df.Date.dt.month < 8]
    
    tmax=np.where(temp.Tmax<0,0,np.where(temp.Tmax>30,30,temp.Tmax)).astype(float)
    tmin=np.where(temp.Tmin<0,0,np.where(temp.Tmin>30,30,temp.Tmin)).astype(float)
    
    ### Delete between the # if you have a summer annual crop
    values2 =((tmax+tmin)/2)
    df['Tmax_GDD_jan1']=(values2)-temp.Tbase
    df['cum_Tmax_GDD_jan1'] = df.Tmax_GDD_jan1.cumsum()
    
    temp2 = df[df.Date.dt.month >= 3]
    temp2 = temp2[temp2.Date.dt.month < 8]
    
    tmax=np.where(temp2.Tmax<0,0,np.where(temp2.Tmax>30,30,temp2.Tmax)).astype(float)
    tmin=np.where(temp2.Tmin<0,0,np.where(temp2.Tmin>30,30,temp2.Tmin)).astype(float)
    
    values2 =((tmax+tmin)/2)
    df['Tmax_GDD_mar1']=(values2)-temp2.Tbase
    df['cum_Tmax_GDD_mar1'] = df.Tmax_GDD_mar1.cumsum()
    ### Delete between the # if you have a summer annual crop
 
    
    return df


In [64]:
df = GDD3(df)
df.head(2)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,...,GDD2_jan1,cum_GDD2_jan1,GDD2_mar1,cum_GDD2_mar1,Tmax_GDD,cum_Tmax_GDD,Tmax_GDD_jan1,cum_Tmax_GDD_jan1,Tmax_GDD_mar1,cum_Tmax_GDD_mar1
0,2019-06-01,Hutchinson 10SW,13.61,28.66,6.186724,20.804979,14.618255,0,21.135,21.135,...,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135,21.135
1,2019-06-02,Hutchinson 10SW,15.11,30.42,6.180718,20.816324,14.635606,0,22.765,43.900,...,22.765,43.900,22.765,43.900,22.555,43.690,22.555,43.690,22.555,43.690


<a name="BJE_step8"></a>
# Step 8: Calculate Pdays 
The next cell defines a function to calculate physiological days. Physiological days incorporates an optimum and maximum growing temperature to determine growth (Saiyed et al. 2009).

In [65]:
def Pdays(df):
    """
    Calculates an individual and cumulative value for Pdays for each day in the data frame
    
    Input: Pandas data frame. Minimum required columns include:
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        Tbase= base growing temperature for given crop
        topt= optimum growing temperature for given crop
        tmax= maximum growing temperature for given crop
        Date = date of collection
        
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        Pdays= The Pday value for that individual day
        cum_Pdays = The cumulative Pdays value for all days in the designated growing period.
    
    Byron Evers
    
    2019-04-06
    """
    T1=np.where(df.Tmin > tbase, df.Tmin, tbase)
    T4=np.where(df.Tmax > 30, 30, df.Tmax)
    T2=((2*T1) + T4)/2
    T3=((2*T4) + T1)/2
    T =((df.Tmax+df.Tmin)/2)
    df['P'] = np.where(T <= topt,(k*(1-((T-topt)*2)/((topt-df.Tmin)*2))),T)
    df['P'] = np.where(T >= topt,(k*(1-((T-topt)*2)/((df.Tmax-topt)*2))),df.P)
    df['P'] = np.where(T <= tbase,0,df.P)
    df['P'] = np.where(T >= tmax,0,df.P)
    df['Pdays']= (1/24)*((5*df.P*T1)+ (8*df.P*T2) + (8*df.P*T3) + (3*df.P*T4))
    df['cum_Pdays']= df.Pdays.cumsum()
    
    ### Delete between the # if you have a summer annual crop
    pd.options.mode.chained_assignment = None 
    temp = df[df.Date.dt.month < 8]
    T1=np.where(temp.Tmin > tbase, temp.Tmin, tbase)
    T4=np.where(temp.Tmax > 30, 30, temp.Tmax)
    T2=((2*T1) + T4)/2
    T3=((2*T4) + T1)/2
    T =((temp.Tmax+temp.Tmin)/2)
    temp['P'] = np.where(T < topt,(k*(1-((T-topt)*2)/((topt-temp.Tmin)*2))),T)
    temp['P'] = np.where(T > topt,(k*(1-((T-topt)*2)/((temp.Tmax-topt)*2))),temp.P)
    temp['P'] = np.where(T < tbase,0,temp.P)
    temp['P'] = np.where(T >= tmax,0,temp.P)
    df['Pdays_jan1']= (1/24)*((5*temp.P*T1)+ (8*temp.P*T2) + (8*temp.P*T3) + (3*temp.P*T4))
    df['cum_Pdays_jan1']= df.Pdays_jan1.cumsum()
    
    temp2 = df[df.Date.dt.month >= 3]
    temp2 = temp2[temp2.Date.dt.month < 8]
    T1=np.where(temp2.Tmin > tbase, temp2.Tmin, tbase)
    T4=np.where(temp2.Tmax > 30, 30, temp2.Tmax)
    T2=((2*T1) + T4)/2
    T3=((2*T4) + T1)/2
    T =((temp2.Tmax+temp2.Tmin)/2)
    temp2['P'] = np.where(T < topt,(k*(1-((T-topt)*2)/((topt-temp2.Tmin)*2))),T)
    temp2['P'] = np.where(T > topt,(k*(1-((T-topt)*2)/((temp2.Tmax-topt)*2))),temp2.P)
    temp2['P'] = np.where(T < tbase,0,temp2.P)
    temp2['P'] = np.where(T > tmax,0,temp2.P)
    df['Pdays_mar1']= (1/24)*((5*temp2.P*T1)+ (8*temp2.P*T2) + (8*temp2.P*T3) + (3*temp2.P*T4))
    df['cum_Pdays_mar1']= df.Pdays_mar1.cumsum()
    ### Delete between the # if you have a summer annual crop
    
    return df

In [66]:
df=Pdays(df)
df.tail(2) 

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,...,cum_Tmax_GDD_jan1,Tmax_GDD_mar1,cum_Tmax_GDD_mar1,P,Pdays,cum_Pdays,Pdays_jan1,cum_Pdays_jan1,Pdays_mar1,cum_Pdays_mar1
395,2020-06-30,Hutchinson 10SW,23.32,34.76,6.215810,20.980016,14.764207,0,29.040,5792.040,...,3563.065,26.66,3318.695,0.112725,3.975631,1917.034303,3.975631,1203.985586,3.975631,1047.179286
396,2020-07-01,Hutchinson 10SW,23.92,37.07,6.223561,20.978849,14.755288,0,30.495,5822.535,...,3590.025,26.96,3345.655,0.000000,0.000000,1917.034303,0.000000,1203.985586,0.000000,1047.179286


<a name="BJE_BMT"></a>
# Step 9: Calculate BMT
The next several cells defines a function to calculate biometeorological time. The coefficients used for this equation are developed from Robertson (1968). However, these coefficients have been edited as Robertson’s coefficients were developed for spring wheat where vernalization was not needed. Additionally, Robertson’s equation is capped at a 5 point scale. However, I was not able to accomplish this limit. I hypothesize that is due to the length of the winter wheat season comparted to a spring wheat growing season. Rega

The first cell identifies the coefficients for each growth stage. The growth stages used for this project are adapted from Robertson and are defined as follows;

* PE- Planting to Emergence
* EJ- Emergence to Jointing
* JH- Jointing to Heading 
* HS- Heading to Soft Dough
* SR- Soft Dough to Ripe


In [67]:
par = {'a0':0,'a1':0,'a2':0,'b0':0,'b1':0,'b2':0,'d1':0,'d2':0,
       'a0_PE':8.413,'a1_PE':1.005,'a2_PE':0,'b0_PE':0,'b1_PE':0.003512,'b2_PE':0.00005026,'d1_PE':0.000366,'d2_PE':0.000004282,
       'a0_EJ':9.413,'a1_EJ':1.005,'a2_EJ':0,'b0_EJ':0,'b1_EJ':0.003512,'b2_EJ':0.00005026,'d1_EJ':0.000366,'d2_EJ':0.000004282,
       'a0_JH':10.93,'a1_JH':0.9256,'a2_JH':0.06025,'b0_JH':0,'b1_JH':0.0002958,'b2_JH':0,'d1_JH':0.0003943,'d2_JH':0,
       'a0_HS':10.94,'a1_HS':1.389,'a2_HS':1,'b0_HS':0,'b1_HS':0.004458,'b2_HS':0,'d1_HS':0.00003109,'d2_HS':0,
       'a0_SR':14.38,'a1_SR':1.614,'a2_SR':1,'b0_SR':0,'b1_SR':0.007733,'b2_SR':0,'d1_SR':0.0003442,'d2_SR':0,}

In [68]:
def BMT (par,df):
    """
    Calculates an individual and cumulative value for BMT for each day in the data frame
    
    Input: Pandas data frame. Minimum required columns include:
        par = dictonary of coefficents for each stage of development
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        PhotoPeriod=daily photo period
        Date = date of collection
        
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        BMT= The BMT value for that individual day
        cum_BMT = The cumulative BMT value for all days in the designated growing period.
    
    Byron Evers
    
    2019-05-05
    """
    BMT=[0]*len(df)
    cum_BMT=[0]*len(df)
    for i in range(1,len(df)):
        if cum_BMT[i-1] < 1:
            par['a0']=par['a0_PE']
            par['a1']=par['a1_PE']
            par['a2']=par['a2_PE']
            par['b0']=par['b0_PE']
            par['b1']=par['b1_PE']
            par['b2']=par['b2_PE']
            par['d1']=par['d1_PE']
            par['d2']=par['d2_PE']
        elif cum_BMT[i-1] >= 1 and cum_BMT[i-1]< 2:
            par['a0']=par['a0_EJ']
            par['a1']=par['a1_EJ']
            par['a2']=par['a2_EJ']
            par['b0']=par['b0_EJ']
            par['b1']=par['b1_EJ']
            par['b2']=par['b2_EJ']
            par['d1']=par['d1_EJ']
            par['d2']=par['d2_EJ']
        elif cum_BMT[i-1] >= 2 and cum_BMT[i-1]< 2.5:
            par['a0']=par['a0_JH']
            par['a1']=par['a1_JH']
            par['a2']=par['a2_JH']
            par['b0']=par['b0_JH']
            par['b1']=par['b1_JH']
            par['b2']=par['b2_JH']
            par['d1']=par['d1_JH']
            par['d2']=par['d2_JH']
        elif cum_BMT[i-1] >= 2.5 and cum_BMT[i-1]< 3:
            par['a0']=par['a0_HS']
            par['a1']=par['a1_HS']
            par['a2']=par['a2_HS']
            par['b0']=par['b0_HS']
            par['b1']=par['b1_HS']
            par['b2']=par['b2_HS']
            par['d1']=par['d1_HS']
            par['d2']=par['d2_HS']  
        else: 
            par['a0']=par['a0_SR']
            par['a1']=par['a1_SR']
            par['a2']=par['a2_SR']
            par['b0']=par['b0_SR']
            par['b1']=par['b1_SR']
            par['b2']=par['b2_SR']
            par['d1']=par['d1_SR']
            par['d2']=par['d2_SR']
        
        photo =df.PhotoPeriod
        photo= np.where(photo < 0,0,photo)
        tmax =df.Tmax
        tmax= np.where(tmax < 0,0,tmax)
        tmin =df.Tmin
        tmin= np.where(tmin < 0,0,tmin)
        if cum_BMT[i-1] < 0.25:
            v1=1
        else:
            v1=abs((par['a1']*(photo[i]-par['a0'])+(par['a2']*((photo[i]-par['a0'])**2))))
        v2=(par['b1']*(tmin[i]-par['b0'])+(par['b2']*((tmin[i]-par['b0'])**2)))
        v3=(par['d1']*(tmin[i]-par['b0'])+(par['d2']*((tmin[i]-par['b0'])**2)))
        BMT[i]=(v1*(v2+v3))

        T=cum_BMT[i-1]+BMT[i]
        cum_BMT[i]=T
    df['BMT']=BMT  
    df['cum_BMT']=cum_BMT   
    return df

In [76]:
df=BMT(par, df)
df.tail(5)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,...,WEDD_jan1,cum_WEDD_jan1,WEDD_mar1,cum_WEDD_mar1,PGDD,cum_PGDD,PGDD_jan1,cum_PGDD_jan1,PGDD_mar1,cum_PGDD_mar1
392,2020-06-27,Hutchinson 10SW,21.21,34.29,6.194789,20.980432,14.785643,0,27.750,5707.565,...,55.535959,5461.704894,55.535959,5380.483441,49.495975,7500.095906,49.495975,4748.667114,49.495975,4697.553415
393,2020-06-28,Hutchinson 10SW,20.40,34.15,6.201415,20.980806,14.779391,0,27.275,5734.840,...,53.756344,5515.461237,53.756344,5434.239785,47.895879,7547.991785,47.895879,4796.562993,47.895879,4745.449293
394,2020-06-29,Hutchinson 10SW,22.78,33.54,6.208425,20.980668,14.772243,0,28.160,5763.000,...,57.093928,5572.555165,57.093928,5491.333713,50.852548,7598.844333,50.852548,4847.415541,50.852548,4796.301841
395,2020-06-30,Hutchinson 10SW,23.32,34.76,6.215810,20.980016,14.764207,0,29.040,5792.040,...,60.506151,5633.061316,60.506151,5551.839863,53.871401,7652.715734,53.871401,4901.286942,53.871401,4850.173242
396,2020-07-01,Hutchinson 10SW,23.92,37.07,6.223561,20.978849,14.755288,0,30.495,5822.535,...,66.351590,5699.412906,66.351590,5618.191453,59.051056,7711.766790,59.051056,4960.337998,59.051056,4909.224299


# Step 10: Calculate Wang and Engel (1998) $GDD_{WE}$ and Photo Degree Days (PGDD)

In [74]:
def WEDD(df):
    """
    Calculates an individual and cumulative value for GDD for each day in the data frame as descirbe by Wang and Engel 1998
    
    Input: Pandas data frame. Minimum required columns include:
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        Date = date of collection
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        WEDD= The growing degree day value for that individual day
        cum_WEDD = The cumulative growing degree day value for all days in the designated growing period.
    
    Byron Evers
    
    2021-04-26
    """
    
    tmin=0
    topt=27.7
    tmax=40
    alpha= (np.log(2))/np.log((tmax-tmin)/(topt-tmin)).astype(float)
    
    tavg =((df.Tmax+df.Tmin)/2).astype(float)
    values = np.where(tavg < tbase, df.Tbase, tavg).astype(float)
    
    #num=2(values-tmin)**alpha(topt-tmin)**alpha-(values-tmin)**(2*alpha).astype(float)
    #den=(topt-tmin)**(2*alpha).astype(float)
    num=2*(values-tmin)**alpha*((topt-tmin)**alpha)-((values-tmin)**alpha)
    den=(topt-tmin)**(2*alpha)
    WEDD=(num/den)*(topt-tmin)
    df['WEDD']=WEDD
    df['cum_WEDD']= df.WEDD.cumsum()
    
    
    temp = df[df.Date.dt.month < 8]
    tavg2 =((temp.Tmax+temp.Tmin)/2).astype(float)
    values2 = np.where(tavg2 < tbase, tbase, tavg2).astype(float)
    num2=2*(values2-tmin)**alpha*((topt-tmin)**alpha)-((values2-tmin)**alpha)
    den2=(topt-tmin)**(2*alpha)
    WEDD2=(num2/den2)*(topt-tmin)
    temp['WEDD_jan1']=WEDD2
    temp['cum_WEDD_jan1']= temp.WEDD_jan1.cumsum()
    temp = temp[['Date', 'WEDD_jan1','cum_WEDD_jan1']].copy()
    df=(pd.merge(df, temp, how='outer', on='Date'))
    
    temp2 = df[df.Date.dt.month >= 3]
    temp2 = temp2[temp2.Date.dt.month < 8]
    tavg3 =((temp2.Tmax+temp2.Tmin)/2).astype(float)
    values3 = np.where(tavg3 < tbase, tbase, tavg3).astype(float)
    num3=2*(values3-tmin)**alpha*((topt-tmin)**alpha)-((values3-tmin)**alpha)
    den3=(topt-tmin)**(2*alpha)
    WEDD3=(num3/den3)*(topt-tmin)
    temp2['WEDD_mar1']=WEDD3
    temp2['cum_WEDD_mar1']= temp2.WEDD_mar1.cumsum()
    temp2 = temp2[['Date', 'WEDD_mar1','cum_WEDD_mar1']].copy()
    df=(pd.merge(df, temp2, how='outer', on='Date'))
    
    PP = 1-(.002*2)*(20-df.PhotoPeriod)**(2)
    df['PGDD']= df.WEDD*PP
    df['cum_PGDD']= df.PGDD.cumsum()
    df['PGDD_jan1']= df.WEDD_jan1*PP
    df['cum_PGDD_jan1']= df.PGDD_jan1.cumsum()
    df['PGDD_mar1']= df.WEDD_mar1*PP
    df['cum_PGDD_mar1']= df.PGDD_mar1.cumsum()
    
    return df

In [77]:
df = WEDD(df)
df.tail(5)

AttributeError: 'DataFrame' object has no attribute 'WEDD_jan1'

In [78]:
df.tail(5)

,Date,STATION,Tmin,Tmax,Sunrise,Sunset,PhotoPeriod,Tbase,GDD,cum_GDD,...,WEDD_jan1,cum_WEDD_jan1,WEDD_mar1,cum_WEDD_mar1,PGDD,cum_PGDD,PGDD_jan1,cum_PGDD_jan1,PGDD_mar1,cum_PGDD_mar1
392,2020-06-27,Hutchinson 10SW,21.21,34.29,6.194789,20.980432,14.785643,0,27.750,5707.565,...,55.535959,5461.704894,55.535959,5380.483441,49.495975,7500.095906,49.495975,4748.667114,49.495975,4697.553415
393,2020-06-28,Hutchinson 10SW,20.40,34.15,6.201415,20.980806,14.779391,0,27.275,5734.840,...,53.756344,5515.461237,53.756344,5434.239785,47.895879,7547.991785,47.895879,4796.562993,47.895879,4745.449293
394,2020-06-29,Hutchinson 10SW,22.78,33.54,6.208425,20.980668,14.772243,0,28.160,5763.000,...,57.093928,5572.555165,57.093928,5491.333713,50.852548,7598.844333,50.852548,4847.415541,50.852548,4796.301841
395,2020-06-30,Hutchinson 10SW,23.32,34.76,6.215810,20.980016,14.764207,0,29.040,5792.040,...,60.506151,5633.061316,60.506151,5551.839863,53.871401,7652.715734,53.871401,4901.286942,53.871401,4850.173242
396,2020-07-01,Hutchinson 10SW,23.92,37.07,6.223561,20.978849,14.755288,0,30.495,5822.535,...,66.351590,5699.412906,66.351590,5618.191453,59.051056,7711.766790,59.051056,4960.337998,59.051056,4909.224299


# Step 11: Export CSV for each growing season

In [ ]:
df.to_csv("/working_directory/df20_rn.csv", index=False)

<a name="Update__graph"></a>

<a name="BJE_policy"></a>
## Permitted Use
Data accessed through http://mesonet.k-state.edu is free for public use and download. Please properly cite the source when sharing, publishing or otherwise disseminating Kansas Mesonet data. All data is preliminary and subject to change due to quality control/assessment procedures. Use at your own risk.

## Unpermitted Use
Automated page scraping or data ingesting without written consent from the Kansas Mesonet is prohibited. Data provided is preliminary and may change. Therefore, authorization is required to prevent unauthorized use and dependencies.

Questions? Contact us at kansas-wdl@k-state.edu

## Citation
Cite the source any time Kansas Mesonet data is published or shared. Not only does this direct your audience to the source, it also helps us track, fund and maintain our network.

We recommended an American Meteorological Society style citation. For example:
Kansas Mesonet, 2017: Kansas Mesonet Historical Data. Accessed 30 June 2017, http://mesonet.k-state.edu/weather/historical

This includes

Date the data was accessed
Webpage title
Webpage URL
Need more information?

Location: Manhattan, Kansas
Publication date is present date


<a name="BJE_ref"></a>
## References
Kyratzis Angelos C., Skarlatos Dimitrios P., Menexes George C., Vamvakousis Vasileios F., Katsiotis Andreas, 2017. Assessment of Vegetation Indices Derived by UAV Imagery for Durum Wheat Phenotyping under a Water Limited and Heat Stressed Mediterranean Environment. Frontiers in Plant Science V.8 P 1114

Miller P, Lanier W, Brandt S (2001) Using Growing Degree Days to Predict
Plant Stages, Ag/Extension Communications Coordinator, Communications
Services, Montana State University-Bozeman, Bozeman, MO, pp 1-8.
Available online:
http://msuextension.org/publications/AgandNaturalResources/MT200103AG.
pdf

Robertson, G. W. 1968. A biometeorological time scale for a
cereal crop involving day and night temperatures and photoperiod.
Int. J. Biometeor. 12: 191223.

Saiyed, I., Bullock, P.R., Sapirstein, H.D., Finlay, G.J., Jarvis, C.K., 2009. Thermal time models for estimating wheat phenological development and weather-based relationships to wheat quality. Can. J. Plant Sci. 89, 429–439.

